<a href="https://colab.research.google.com/github/Alan-Marcus/HCCS/blob/main/03%20Analysis%20scripts%20and%20templates/SLEAP/Colab%20Notebooks/SLEAP_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HCCS: Training and Inference with SLEAP**

This is an example notebook to demonstrate the use of SLEAP (https://sleap.ai) with videos recorded using the Home-Cage Camera System (HCCS; https://github.com/Alan-Marcus/HCCS).
This notebook will use your labelled data to train a top-down model to identify the centroid location on an animal, then predict (infer) the centroids of all animals across all frames of your videos and link them into individual tracks.

#0. Setup

1. Upload the configuration and video files to their respective folders in Google Drive.
2. Make sure Colab is set to use a GPU (menu Runtime > Change runtime type: Hardware accelerator is set to GPU).
3. Run the following cells to install SLEAP and connect your Google Drive to this session.

In [ ]:
#@title Install SLEAP
import time
from google.colab import files
from google.colab import drive
!pip uninstall -y opencv-python opencv-contrib-python
!pip install sleap

In [ ]:
#@title Connect Google Drive
drive.mount('/content/drive/')

#1. Specify Training File and Model Details


In [ ]:
#@title Edit Parameters
path_to_SLEAP_folder = "/content/drive/MyDrive/SLEAP" #@param {type:"string"}
labelled_frames_file = "HCCS_example_project.pkg.slp" #@param {type:"string"}
model_name = "HCCS_example" #@param {type:"string"}

#---don't edit below
centroid_config = path_to_SLEAP_folder + "/config/centroid.json"
centered_instance_config = path_to_SLEAP_folder + "/config/centered_instance.json"
train_pkg = path_to_SLEAP_folder + "/config/" + labelled_frames_file
models = path_to_SLEAP_folder + "/models/" + model_name
centroid_model = models + ".centroid"
centered_instance_model = models + ".centered_instance"
video_folder = path_to_SLEAP_folder + "/videos/"

print("Parameters set")
print("SLEAP folder: " + path_to_SLEAP_folder)
print("config folder: " + path_to_SLEAP_folder + "/config/")
print("models folder: " + path_to_SLEAP_folder + "/models/")
print("videos folder: " + video_folder)
print("\n" + "Labelled frames file: " + train_pkg)
print("\n" + "Models")
print("Centroid model: " + centroid_model)
print("Centered Instance model: " + centered_instance_model)


#2. Train Model

In [ ]:
#@title Train Centroid Model
!sleap-train $centroid_config $train_pkg --run_name $models

time.sleep(5)
print("Done")

In [ ]:
#@title Train Centered Instance Model
!sleap-train $centered_instance_config $train_pkg --run_name $models

time.sleep(5)
print("Done")

#3. Use Trained Model for Inference
If using a model trained in a previous session, complete steps 0 and 1 above before continuing here.

In [ ]:
#@title Add List of Videos for Inference
#@markdown (Paste contents of text file created by "HCCS_SLEAP-create-training-videos-list.bat" here)
videos = "PiCam01-Air_20210122_09-35.mp4 PiCam01-Air_20210122_10-35.mp4 PiCam02-Cl2_20210122_09-35.mp4 PiCam02-Cl2_20210122_10-35.mp4" #@param {type:"string"}

videos_list = list(videos.split(" "))
print("Videos in list:", len(videos_list))
print("Videos list: ", videos_list)

#@markdown ---
#@markdown ### Choose to infer for all frames in each video or a selection
#@markdown (Choose "Selection" and enter either a range (e.g. 200-1200) or individual frames (e.g. 200,230,260,800,850). Choosing "All" ignores inference_selection)
infer_on_frames = "All" #@param ["All", "Selection"]
inference_selection = "200-1200" #@param {type:"string"}

if infer_on_frames == "Selection":
  frames_opt = "--frames " + inference_selection
  print("Inferring for frames: " + inference_selection)
else:
  frames_opt = ""
  print("Inferring for all frames")

#@markdown ---
#@markdown ### Adjust tracking parameters
#@markdown (Setting tracking_cull to 0 will disable culling)
tracking_cull = 5#@param {type:"number"}
tracking_window = 10#@param {type:"number"}

if tracking_cull > 0:
  cull = "--tracking.target_instance_count " + str(tracking_cull) + " --tracking.pre_cull_to_target 1"
  print("Culling the number of individuals to track to a maximum of " + str(tracking_cull))
else:
  cull = "--tracking.pre_cull_to_target 0"
  print("Track culling disabled")

for video in videos_list:
  vid = video_folder + video
  pred = vid + ".predictions.slp"
  print(vid, "\n")
  !sleap-track $vid $frames_opt --tracking.tracker simple --tracking.similarity centroid --tracking.track_window $tracking_window --tracking.match hungarian $cull --tracking.post_connect_single_breaks 1 -m $centered_instance_model -m $centroid_model
  print("----------")
  !sleap-inspect $pred
  print("----------")
  print("----------", "\n")
print("Done")

#4. Download output

In [ ]:
#@title Zip the prediction files
predictions_zip_file_name = "HCCS_example_project_predictions_iteration-01" #@param {type:"string"}
!cd $video_folder && zip -r $path_to_SLEAP_folder/$predictions_zip_file_name *.slp
print("Done")

In [ ]:
#@title Download the zipped predictions
files.download(path_to_SLEAP_folder + "/" + predictions_zip_file_name + ".zip")

In [ ]:
#@title Zip the models folder
models_zip_file_name = "HCCS_example_project_models_iteration-01" #@param {type:"string"}
!cd $path_to_SLEAP_folder/models/ && zip -r $path_to_SLEAP_folder/$models_zip_file_name *.*
print("Done")

In [ ]:
#@title Download the zipped models folder
files.download(path_to_SLEAP_folder + "/" + models_zip_file_name + ".zip")